In [24]:
import json
from datetime import datetime

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 

In [25]:
%cd ../data
np.random.seed(42)

/Users/alee/Desktop/westford14-ETL-voucher/data


### Initial Data Exploration

In [26]:
df = pd.read_parquet("data.parquet.gzip")

In [27]:
df.head()

,timestamp,country_code,last_order_ts,first_order_ts,total_orders,voucher_amount
0,2020-05-20 15:43:38.364972+00:00,China,2020-04-19 00:00:00+00:00,2020-04-18 00:00:00+00:00,0.0,5720.0
1,2020-05-20 15:43:47.914346+00:00,China,2020-04-19 00:00:00+00:00,2020-04-13 00:00:00+00:00,1.0,8800.0
2,2020-05-20 15:36:56.866239+00:00,China,2020-04-19 00:00:00+00:00,2020-04-15 00:00:00+00:00,0.0,1760.0
3,2020-05-20 15:00:33.694108+00:00,China,2020-04-19 00:00:00+00:00,2020-04-13 00:00:00+00:00,,1760.0
4,2020-05-20 15:43:18.521402+00:00,China,2020-04-19 00:00:00+00:00,2020-04-14 00:00:00+00:00,3.0,8800.0


 Subset the data down to peru -- ensuring that all the "Peru" country codes are not mangled 

In [28]:
df['country_code'].value_counts() 

China        169722
Australia    134004
Peru         106547
Latvia       101154
Name: country_code, dtype: int64

In [29]:
### no country codes are null -- can subset to just 'Peru'
df[pd.isnull(df['country_code'])]

,timestamp,country_code,last_order_ts,first_order_ts,total_orders,voucher_amount


In [30]:
df = df[df['country_code'].isin(['Peru'])]

In [31]:
for col in df.columns:
    print(f"{col}: null-values - {len(df[pd.isnull(df[col])])}" )

timestamp: null-values - 0
country_code: null-values - 0
last_order_ts: null-values - 0
first_order_ts: null-values - 0
total_orders: null-values - 0
voucher_amount: null-values - 13950


Based on above -- we will need to do the most amount of cleaning for nulls on the `voucher_amount` column

In [32]:
# there are ~9000 rows that don't have a total_orders value, so
# for now, we are going to drop 

In [33]:
df = df[~df['total_orders'].isin([''])]

In [34]:
# dropping the null voucher_amounts
df = df[~pd.isnull(df['voucher_amount'])]

In [35]:
len(df)

85150

#### Creation of `frequent_segment`

In [36]:
df['total_orders'] = df['total_orders'].apply(float).apply(int)

In [37]:
def freq_segment(orders: int) -> str:
    if orders < 4:
        return "0-4"
    elif orders < 13:
        return "5-13"
    elif orders < 37:
        return "13-37"
    else:
        return np.nan

In [38]:
df['frequent_segment'] = df['total_orders'].apply(freq_segment)

#### Creation of `recency_segment`

In [39]:
# cast to datetime 
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['last_order_ts'] = pd.to_datetime(df['last_order_ts'])

In [40]:
df['time_diff'] = (df['timestamp'] - df['last_order_ts']).astype('timedelta64[D]').apply(int)

In [41]:
def rec_segment(days: float) -> str:
    if days < 30:
        return np.nan
    elif days < 60: 
        return "30-60"
    elif days < 90:
        return "60-90"
    elif days < 120:
        return "90-120"
    elif days < 180:
        return "120-180"
    else:
        return "180+"

In [42]:
df['recency_segment'] = df['time_diff'].apply(rec_segment)

#### Final Cleaning and Resultant Dict

In [43]:
df = df[['voucher_amount', 'frequent_segment', 'recency_segment']]
cleaned = df[~pd.isnull(df).any(axis=1)]

In [44]:
# find most used voucher per segment
rec_seg = cleaned.groupby(['recency_segment'], as_index=False)['voucher_amount'].agg(lambda x: x.value_counts().index[0])
freq_seg = cleaned.groupby(['frequent_segment'], as_index=False)['voucher_amount'].agg(lambda x: x.value_counts().index[0])

In [45]:
rec_seg

,recency_segment,voucher_amount
0,30-60,2640.0
1,90-120,2640.0


In [46]:
freq_seg

,frequent_segment,voucher_amount
0,0-4,2640.0
1,13-37,2640.0
2,5-13,2640.0


#### Addition of Missing Groups

In [53]:
freq_dict = freq_seg.set_index('frequent_segment').to_dict()['voucher_amount']
for x in ['0-4', '5-13', '13-37']:
    if x not in freq_dict:
        freq_dict[x] = 0

In [55]:
with open("freq_segment.json", "w") as f:
    json.dump(freq_dict, f)

In [56]:
rec_dict = rec_seg.set_index('recency_segment').to_dict()['voucher_amount']
for x in ['30-60', '60-90', '90-120', '120-180', '180+']:
    if x not in rec_dict:
        rec_dict[x] = 0

In [59]:
with open("rec_segment.json", "w") as f:
    json.dump(rec_dict, f)